In [165]:
from langchain_community.document_loaders import YoutubeLoader
from langchain_community.document_loaders.youtube import TranscriptFormat
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFaceEndpoint, ChatHuggingFace
from langchain_core.documents import Document
# from pinecone import Pinecone, ServerlessSpec
# from langchain_pinecone import PineconeVectorStore
import pandas as pd
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from dotenv import find_dotenv, load_dotenv, dotenv_values
from src.keeper import video_comments_to_dataframe, get_channel_videos, create_columnDefs, ds_to_html_table, df_to_html_table, get_channel_statistics
import os, re, time, chromadb, pickle
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_chroma import Chroma
from langchain_groq import ChatGroq
from langchain_community.chat_message_histories import ChatMessageHistory


In [2]:
dotenv_path = find_dotenv(raise_error_if_not_found = True)
load_dotenv(dotenv_path)

True

In [3]:
myYouTube_Token=os.getenv('YOUTUBE_API_KEY')
myLangChain_Token = os.getenv('LANGCHAIN_API_KEY')
# myPineCone_Token = os.getenv('PINECONE_API_KEY')
os.environ["LANGCHAIN_TRACING_V2"] = "true"
# pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [4]:
client = chromadb.PersistentClient(path = "data")

In [40]:
yt_get_channel_urls = ["https://www.youtube.com/@melrobbins", "https://www.youtube.com/@hubermanlab"]


In [41]:
new_video_list_df = None
for channel in yt_get_channel_urls:
    df=pd.DataFrame(get_channel_videos(channel, myYouTube_Token))
    new_video_list_df= pd.concat([new_video_list_df, df.transpose()])

len(new_video_list_df)
# video_list_df = pd.read_csv('data/video_list.csv')

Initializing YouTube API client...
Attempting to get channel ID for URL: https://www.youtube.com/@melrobbins
Searching for channel...
Found channel ID via search: UCk2U-Oqn7RXf-ydPqfSxG5g
Starting to fetch videos for channel ID: UCk2U-Oqn7RXf-ydPqfSxG5g
Fetching page of results. Token: None
Fetched 50 videos. Total so far: 50
Fetching page of results. Token: CDIQAA
Fetched 50 videos. Total so far: 100
Fetching page of results. Token: CGQQAA
Fetched 50 videos. Total so far: 150
Fetching page of results. Token: CJYBEAA
Fetched 50 videos. Total so far: 200
Fetching page of results. Token: CMgBEAA
Fetched 50 videos. Total so far: 249
Fetching page of results. Token: CPoBEAA
Fetched 50 videos. Total so far: 299
Fetching page of results. Token: CKwCEAA
Fetched 50 videos. Total so far: 349
Fetching page of results. Token: CN4CEAA
Fetched 50 videos. Total so far: 398
Fetching page of results. Token: CJADEAA
Fetched 50 videos. Total so far: 448
Fetching page of results. Token: CMIDEAA
Fetched 5

762

In [42]:
get_scripts = video_list_df['url'].tolist()
new_scripts = new_video_list_df['url'].tolist()

In [45]:
# new_video_list_df.to_csv('data/new_video_list.csv')

In [51]:
new_video_list_df

,url,title,publish_date,view_count,like_count,dislike_count,favourite_count,comment_count,duration,watch_time,privacy_status,tags
bVxpV8GSD70,https://www.youtube.com/watch?v=bVxpV8GSD70,The Pep Talk You Need if You’re Actively Worki...,2024-09-03 11:30:30+00:00,2594,330,0,0,56,00:10:05,0,public,"[Mel Robbins, Mel Robbins Motivation, Mel Robb..."
TtK_GroG-5E,https://www.youtube.com/watch?v=TtK_GroG-5E,THE 5 SECOND RULE That Can Change Your Life | ...,2024-09-01 15:30:13+00:00,7864,437,0,0,19,00:00:00,0,public,"[Mel Robbins, Mel Robbins Motivation, Mel Robb..."
i-LH9GMfdP4,https://www.youtube.com/watch?v=i-LH9GMfdP4,This Exercise Will DRASTICALLY Change They Way...,2024-09-01 11:30:20+00:00,11213,686,0,0,49,00:12:44,0,public,"[Mel Robbins, Mel Robbins Motivation, Mel Robb..."
fkIgviO9ZkA,https://www.youtube.com/watch?v=fkIgviO9ZkA,5 Goal Setting Mistakes And How To Fix Them Th...,2024-08-31 15:30:25+00:00,8204,379,0,0,31,00:00:00,0,public,"[Mel Robbins, Mel Robbins Motivation, Mel Robb..."
7Y7x3WhpPo4,https://www.youtube.com/watch?v=7Y7x3WhpPo4,"If you’re expecting fast results, you need to ...",2024-08-31 11:30:07+00:00,9221,691,0,0,42,00:10:58,0,public,"[Mel Robbins, Mel Robbins Motivation, Mel Robb..."
...,...,...,...,...,...,...,...,...,...,...,...,...
NAATB55oxeQ,https://www.youtube.com/watch?v=NAATB55oxeQ,"How to Defeat Jetlag, Shift Work & Sleeplessness",2021-01-25 13:00:18+00:00,512989,13668,0,0,1481,00:00:00,0,public,"[sleep, andrew huberman, huberman lab, dr. and..."
nwSkFq4tyC0,https://www.youtube.com/watch?v=nwSkFq4tyC0,"Using Science to Optimize Sleep, Learning & Me...",2021-01-18 13:00:25+00:00,1353722,37389,0,0,2027,00:00:00,0,public,"[andrew huberman, dr. andrew huberman, sleep, ..."
nm1TxQj9IsQ,https://www.youtube.com/watch?v=nm1TxQj9IsQ,Master Your Sleep & Be More Alert When Awake,2021-01-11 13:59:33+00:00,3896189,111272,0,0,5162,00:00:00,0,public,"[sleep, andrew huberman, dr. andrew huberman, ..."
H-XfCl-HpRM,https://www.youtube.com/watch?v=H-XfCl-HpRM,How Your Nervous System Works & Changes,2021-01-04 16:00:44+00:00,1468960,58644,0,0,3136,00:00:00,0,public,"[learning, learn faster, brain, understand you..."


In [43]:
third_list = [item for item in new_scripts if item not in get_scripts]

In [53]:
merged_df = new_video_list_df.merge(video_list_df, on=['url'], how='left', indicator=True)
result_df = merged_df[merged_df['_merge'] == 'left_only'].drop(columns=['_merge'])

In [54]:
result_df

,url,title_x,publish_date_x,view_count_x,like_count_x,dislike_count_x,favourite_count_x,comment_count_x,duration_x,watch_time_x,...,publish_date_y,view_count_y,like_count_y,dislike_count_y,favourite_count_y,comment_count_y,duration_y,watch_time_y,privacy_status_y,tags_y
0,https://www.youtube.com/watch?v=bVxpV8GSD70,The Pep Talk You Need if You’re Actively Worki...,2024-09-03 11:30:30+00:00,2594,330,0,0,56,00:10:05,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.youtube.com/watch?v=TtK_GroG-5E,THE 5 SECOND RULE That Can Change Your Life | ...,2024-09-01 15:30:13+00:00,7864,437,0,0,19,00:00:00,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://www.youtube.com/watch?v=i-LH9GMfdP4,This Exercise Will DRASTICALLY Change They Way...,2024-09-01 11:30:20+00:00,11213,686,0,0,49,00:12:44,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://www.youtube.com/watch?v=fkIgviO9ZkA,5 Goal Setting Mistakes And How To Fix Them Th...,2024-08-31 15:30:25+00:00,8204,379,0,0,31,00:00:00,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://www.youtube.com/watch?v=7Y7x3WhpPo4,"If you’re expecting fast results, you need to ...",2024-08-31 11:30:07+00:00,9221,691,0,0,42,00:10:58,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,https://www.youtube.com/watch?v=7FOgolNYuX0,8 Things I Wish I Knew in College,2024-08-29 09:00:05+00:00,16925,646,0,0,53,00:44:51,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,https://www.youtube.com/watch?v=-SWemh6Y4VI,Find your purpose in life | Mel Robbins #Shorts,2024-08-28 19:30:00+00:00,25700,3140,0,0,51,00:01:00,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,https://www.youtube.com/watch?v=SQ-ft7C77E8,How to Reprogram the 95% of Your Mind That Con...,2024-08-28 15:30:18+00:00,21283,899,0,0,55,00:00:00,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52,https://www.youtube.com/watch?v=olFIJIr4Ndc,Silence Imposter Syndrome With THESE 4 Words |...,2023-08-17 19:00:20+00:00,40613,2785,0,0,51,00:00:58,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
89,https://www.youtube.com/watch?v=0QI2BQmnsH8,A Yale Psychologist Explains Why Teens are Spi...,2023-05-18 09:00:00+00:00,35654,1123,0,0,76,00:00:00,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
third_list

['https://www.youtube.com/watch?v=bVxpV8GSD70',
 'https://www.youtube.com/watch?v=TtK_GroG-5E',
 'https://www.youtube.com/watch?v=i-LH9GMfdP4',
 'https://www.youtube.com/watch?v=fkIgviO9ZkA',
 'https://www.youtube.com/watch?v=7Y7x3WhpPo4',
 'https://www.youtube.com/watch?v=7FOgolNYuX0',
 'https://www.youtube.com/watch?v=-SWemh6Y4VI',
 'https://www.youtube.com/watch?v=SQ-ft7C77E8',
 'https://www.youtube.com/watch?v=olFIJIr4Ndc',
 'https://www.youtube.com/watch?v=0QI2BQmnsH8',
 'https://www.youtube.com/watch?v=G_XhrNkJmEg',
 'https://www.youtube.com/watch?v=rOFrrpOU1MM',
 'https://www.youtube.com/watch?v=d1vyk_EnRy4',
 'https://www.youtube.com/watch?v=zMu_ztLm_qo',
 'https://www.youtube.com/watch?v=_rL96QUnl8M',
 'https://www.youtube.com/watch?v=2x0oNObZMjs',
 'https://www.youtube.com/watch?v=Tq4xfxIUgQ4',
 'https://www.youtube.com/watch?v=Tikrn8ZaKQA',
 'https://www.youtube.com/watch?v=spPHEYwqjsQ',
 'https://www.youtube.com/watch?v=GQmr-eF9i0E',
 'https://www.youtube.com/watch?v=U2BPit

In [9]:
# index_name = "keeper-index"  # change if desired

# existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

# if index_name not in existing_indexes:
#     pc.create_index(
#         name=index_name,
#         dimension=384,
#         metric="cosine",
#         spec=ServerlessSpec(cloud="aws", region="us-east-1"),
#     )
#     while not pc.describe_index(index_name).status["ready"]:
#         time.sleep(1)

# index = pc.Index(index_name)
# vector_store = PineconeVectorStore(index=index, embedding=embedding_model)
## Creating the Chroma collection
collection = client.get_or_create_collection(name="keeper_collection", metadata={"hnsw:space": "cosine"})


In [10]:
vector_store_from_client = Chroma(
    client=client,
    collection_name="keeper_collection",
    embedding_function=embedding_model,
)

# loading the transcripts 

In [47]:
def load_transcript(url):
    loader = YoutubeLoader.from_youtube_url(url, translation="en", add_video_info=True, transcript_format=TranscriptFormat.CHUNKS, chunk_size_seconds=30)
    time.sleep(3)  # Pause for 3 seconds
    transcript = loader.load()
    return transcript
    

# Splitting the transcripts into chunks

In [48]:
def split_transcript(transcript, chunk_size=1000, chunk_overlap=200):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, add_start_index=True, chunk_overlap=chunk_overlap)
    trans = text_splitter.split_documents(transcript)
    return trans

In [49]:
transcripts = []
c = 0
x = 0
p = 0
z = 0
myrecords = 0
for script in third_list:
    myrecords += 1
    if myrecords % 100 == 0:
        print(f"processing record {myrecords}")
        
    try:
        transcript = load_transcript(script)
    except Exception as e:
        print(f"There was an error processing transcript record:{myrecords} : ", e)
        c+=1
        continue #Skip to the next script
    try: 
        chunks = split_transcript(transcript)
        author = chunks[0].metadata["author"].replace(" ","_")
    except Exception as e:
        print(f"There was an error processing chunk record:{myrecords} : ", e)
        x+=1
        continue #Skip to the next script
    try:
        texts = [doc.page_content for doc in chunks] # necessary for chromadb
        text_keys = [doc.metadata['source'].split('=')[1].split('&')[0] for doc in chunks]
        indexed_keys = [text_keys[i]+"_" + str(i)  for i in range(len(text_keys))]
        # embeddings = embedding_model.embed_documents(texts) 
        meta_data = [doc.metadata for doc in chunks] #necessary for chromadb
    except Exception as e:
        print(f"There was an error processing index record:{myrecords} : ", e)
        p+=1
        continue #Skip to the next script
    try:
        # vector_store.add_documents(documents=chunks, ids=indexed_keys)  #pinecone
        collection.add(documents = texts, ids=indexed_keys, metadatas=meta_data) #change documents = chunks for pinecone, documemnts = texts for chromadb
    except Exception as e:
        print(f"There was an error storing record:{myrecords} : ", e)
        z+=1
        continue #Skip to the next script
    transcripts.append(transcript)
    

print (f'count of transcript errors: {c}')
print (f'count of chunk errors: {x}')
print (f'count of text errors: {p}')
print (f'count of storing errors: {z}')   

count of transcript errors: 0
count of chunk errors: 0
count of text errors: 0
count of storing errors: 0


In [50]:
with open('data/new_transcripts.pkl', 'wb') as file:
   pickle.dump(transcript, file)
        

In [161]:
result['documents']

[["going to torture me and tie my hand  behind my back and make me like do this  i could do it  it would look like i was writing with my  foot  and if lewis came up to me and said hey  mel you want some bulletproof coffee i'd  be like louis i'm trying to concentrate  i can't do this  my prefrontal cortex would be el fuego  because it is  firing on all cylinders to communicate  to my hand new behavior  so  the thing that's cool about that  is that you can use a simple trick  the moment you feel yourself hesitate  the moment you've got one of those  moments where you know that you need to  this is that moment that lewis talks to  you about where you got to step outside  of your comfort zone and you've got to  lean into your passion and you've got to  really take some risks and you've got to  feel the fear and you've got to do it  anyway  that's the moment where you just woke up  and now you got a decision to make are  you going to drift back into the habits  or are you going to awaken",


In [162]:
seen_titles=set()
unique_titles = []
for sublist in result['metadatas']:
    for record in sublist:
        title = record['title']
        if title not in seen_titles:
            unique_titles.append(title)
            seen_titles.add(title)
    

# titles = [record['title'] for sublist in result['metadatas'] for record in sublist]
# titles = [d['title'] for sublist in data for d in sublist]
seen_titles

{'Create Success In Your Life - 4 Hours of Wisdom To Listen To All Day | Mel Robbins',
 'Dare To Dream Big, Your LIFE Depends On It | Mel Robbins',
 'If You Do THIS, EVERYTHING Will Change For You | Mel Robbins',
 'Simple Daily Habits That Will Change Your Life | Mel Robbins',
 'Stop Living Your Life On AUTOPILOT, Do This INSTEAD! | Mel Robbins',
 'The Science of Setting & Achieving Goals',
 'To Turn Your Dream Into A Reality, Watch This | Mel Robbins',
 'Until You Answer THIS Question, You Will CONTINUE To Be Stuck! | Mel Robbins'}

In [163]:
result['distances']

[[0.5418909788131714,
  0.5449168086051941,
  0.5462613105773926,
  0.5483083724975586,
  0.5514287948608398,
  0.560256838798523,
  0.565642237663269,
  0.5666981935501099,
  0.5684000253677368,
  0.5830892324447632]]

In [164]:
chat = ChatGroq(temperature = 0, groq_api_key = os.getenv('GROQ_API_KEY'), model_name = "llama3-70b-8192")
# llm = HuggingFaceEndpoint(
#     repo_id="meta-llama/Meta-Llama-3.1-8B-Instruct",
#     task="text-generation",
#     max_new_tokens=512,
#     do_sample=False,
#     repetition_penalty=1.03,
# )

# chat = ChatHuggingFace(llm=llm, verbose=True)

In [166]:
chat_history = ChatMessageHistory()
chat_history(

In [169]:
system = "You are a helpful life coach and expert in neurology.  All of your recommendations are backed by science. "
human = "How do I train myself to take action: {context}"
result = collection.query(query_texts=human)
context = result['documents'].insert(0," you can use the following for additional context ")
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

chain = prompt | chat
response = chain.invoke({"system": system, "human": human, "context": context})


In [177]:
this = " ".join(result['documents'][1])
this

"going to torture me and tie my hand  behind my back and make me like do this  i could do it  it would look like i was writing with my  foot  and if lewis came up to me and said hey  mel you want some bulletproof coffee i'd  be like louis i'm trying to concentrate  i can't do this  my prefrontal cortex would be el fuego  because it is  firing on all cylinders to communicate  to my hand new behavior  so  the thing that's cool about that  is that you can use a simple trick  the moment you feel yourself hesitate  the moment you've got one of those  moments where you know that you need to  this is that moment that lewis talks to  you about where you got to step outside  of your comfort zone and you've got to  lean into your passion and you've got to  really take some risks and you've got to  feel the fear and you've got to do it  anyway  that's the moment where you just woke up  and now you got a decision to make are  you going to drift back into the habits  or are you going to awaken way 

In [170]:
print(response.content, end='\n')
print(response.usage_metadata)

My friend, taking action can be a challenging task, especially when procrastination and self-doubt creep in. But fear not, for I've got some evidence-based strategies to help you train yourself to take action!

**1. Break it down into smaller, manageable tasks (Task Segmentation)**

When a task seems too big, our brains can get overwhelmed, leading to inaction. Break down the task into smaller, achievable steps. This will make it feel less daunting and more manageable, increasing the likelihood of taking action. (Source: "Task Segmentation" by Amabile, 1993)

**2. Create an implementation intention (If-Then Planning)**

Identify the specific actions you want to take and create an if-then plan. For example, "If it's Monday, then I will work on Project X for 30 minutes." This strategy helps to bridge the gap between intention and action. (Source: "If-Then Planning" by Gollwitzer & Sheeran, 2006)

**3. Use the 2-minute rule**

If a task can be done in less than 2 minutes, do it immediatel

In [160]:
result['metadatas']

[[{'author': 'Mel Robbins',
   'description': 'Unknown',
   'length': 15067,
   'publish_date': '2022-05-24 00:00:00',
   'source': 'https://www.youtube.com/watch?v=ubYzQ_j-TGE&t=0s',
   'start_seconds': 0,
   'start_timestamp': '00:00:00',
   'thumbnail_url': 'https://i.ytimg.com/vi/ubYzQ_j-TGE/hq720.jpg',
   'title': 'Create Success In Your Life - 4 Hours of Wisdom To Listen To All Day | Mel Robbins',
   'view_count': 10941},
  {'author': 'Mel Robbins',
   'description': 'Unknown',
   'length': 6086,
   'publish_date': '2022-04-14 00:00:00',
   'source': 'https://www.youtube.com/watch?v=QJp0wq6K9Rs&t=0s',
   'start_seconds': 0,
   'start_timestamp': '00:00:00',
   'thumbnail_url': 'https://i.ytimg.com/vi/QJp0wq6K9Rs/hq720.jpg',
   'title': 'Dare To Dream Big, Your LIFE Depends On It | Mel Robbins',
   'view_count': 25241},
  {'author': 'Mel Robbins',
   'description': 'Unknown',
   'length': 4036,
   'publish_date': '2022-05-01 00:00:00',
   'source': 'https://www.youtube.com/watch?v

In [66]:
collection.

AttributeError: 'Collection' object has no attribute 'last'

In [ ]:
texts = [doc.page_content for doc in chunks]
text_keys = [doc.metadata['source'].split('=')[1].split('&')[0] for doc in chunks]
indexed_keys = [text_keys[i]+"_" + str(i)  for i in range(len(text_keys))]
embeddings = embedding_model.embed_documents(texts) 

In [ ]:
meta_data

# Embedding the Transcripts

In [ ]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")



In [ ]:
texts = [doc.page_content for doc in chunks]

In [ ]:
n=0

In [ ]:
text_keys = [doc.metadata['source'].split('=')[1].split('&')[0] for doc in chunks]

In [ ]:
indexed_text_keys = [text_key+str(i+1) for i, text_key in enumerate(text_keys)]

In [ ]:
indexed_keys

In [ ]:
embeddings = embedding_model.embed_documents(texts)

In [ ]:
chunks[0].metadata['author']

# Store the created embeddings to Pinecone 

In [ ]:
pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [ ]:
results = vector_store.similarity_search(query="How to achieve your goals and not get discouraged",k=10)
for doc in results:
    print(f"* {doc.page_content} [{doc.metadata}]")

In [ ]:
for doc in results:
    print(f'* {[doc.metadata['author']]}')

In [ ]:
import pprint as pp

In [ ]:
result

In [ ]:
client.delete_collection('keeper_collection')

In [156]:
!pip install langgraph

   ---------------------------------------- 0.0/91.1 kB ? eta -:--:--
   ---- ----------------------------------- 10.2/91.1 kB ? eta -:--:--
   ----------------- ---------------------- 41.0/91.1 kB 653.6 kB/s eta 0:00:01
   ----------------- ---------------------- 41.0/91.1 kB 653.6 kB/s eta 0:00:01
   ---------------------------------------- 91.1/91.1 kB 518.1 kB/s eta 0:00:00
